In [1]:
import pandas as pd
import geopandas as gpd
import psycopg2 as psql
import numpy as np

In [24]:
def fetch_table(query, connect):
    '''
    Fetch table from database
    Input: str, str / Output: Dataframe
    '''
    cur = connect.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    df = pd.DataFrame(rows)
    cur.close()
    return df

In [25]:
db_name = 'SubwayDWT'
conn = psql.connect(host = 'localhost', port = 5432, user = 'postgres', password = 'user', database = db_name)
cur = conn.cursor()

In [26]:
query = 'SELECT * FROM subwayinfo_x'
raw_x = fetch_table(query, conn)

In [27]:
conn.close()

In [28]:
raw_x.rename(columns = {0: 'id', 3: 'name'}, inplace = True)

In [2]:
raw_y = pd.read_csv('../RawData/서울교통공사_역별 일별 시간대별 승하차인원 정보_20231031.csv', encoding = 'cp949')
raw_y9 = pd.read_csv('../RawData/서울교통공사_9호선2_3단계 역별일별시간대별승하차인원_20231031.csv', encoding = 'cp949')

raw_y9['06시이전'] = raw_y9['03-04시간대'] + raw_y9['04-05시간대'] + raw_y9['05-06시간대']
raw_y9['24시이후'] = raw_y9['00-01시간대'] + raw_y9['01-02시간대'] + raw_y9['02-03시간대']
raw_y9.drop(columns = ['03-04시간대','04-05시간대', '05-06시간대','00-01시간대', '01-02시간대', '02-03시간대'], inplace = True)

raw_y9['승하차구분'] = raw_y9['승하차구분'].str.replace('순승차', '승차')
raw_y9['승하차구분'] = raw_y9['승하차구분'].str.replace('순하차', '하차')

raw_y = pd.concat([raw_y, raw_y9])

In [3]:
new_rawy = raw_y.loc[:, ['수송일자', '역명', '승하차구분', '06시이전', '06-07시간대',
       '07-08시간대', '08-09시간대', '09-10시간대', '10-11시간대', '11-12시간대', '12-13시간대',
       '13-14시간대', '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대',
       '19-20시간대', '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대', '24시이후']].groupby(['수송일자', '역명', '승하차구분']).sum().reset_index()

new_rawy = new_rawy.rename(columns = {'역명': 'name'})
# Join line info and transfer
def station_name_cut(x):
    x = x.split('(')[0]
    if x[-1] == '역':
        return x[:-1]
    else: return x
new_rawy['name'] = new_rawy['name'].apply(station_name_cut)

In [4]:
new_rawy

,수송일자,name,승하차구분,06시이전,06-07시간대,07-08시간대,08-09시간대,09-10시간대,10-11시간대,11-12시간대,...,15-16시간대,16-17시간대,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후
0,2023-01-01,가락시장,승차,100,131,186,218,247,310,302,...,516,435,432,400,349,294,240,208,61.0,3
1,2023-01-01,가락시장,하차,29,154,134,253,284,323,329,...,397,513,552,462,350,262,255,210,174.0,59
2,2023-01-01,가산디지털단지,승차,82,94,132,192,276,251,332,...,930,1033,995,829,662,537,586,212,55.0,0
3,2023-01-01,가산디지털단지,하차,23,99,110,256,418,532,627,...,846,655,541,420,320,327,284,209,146.0,6
4,2023-01-01,강남,승차,1015,909,592,506,585,697,798,...,1591,1792,2003,1965,1787,1968,2411,2049,1082.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151973,2023-10-31,화랑대,하차,39,124,348,953,675,412,391,...,469,588,960,1588,1396,721,729,556,401.0,168
151974,2023-10-31,회현,승차,58,132,240,379,476,593,1060,...,2828,3356,4440,5363,2420,2127,964,583,219.0,33
151975,2023-10-31,회현,하차,25,909,2289,6810,4315,3222,2972,...,1554,1155,864,753,407,238,228,194,143.0,56
151976,2023-10-31,효창공원앞,승차,93,242,822,1260,663,365,342,...,390,535,657,1056,446,303,249,179,69.0,26


In [19]:
weekdays = ['2023-10-16', '2023-10-17','2023-10-18','2023-10-19','2023-10-20']
weekday_y = new_rawy[new_rawy['수송일자'].isin(weekdays)]
weekday_avg_y = weekday_y.groupby(['name', '승하차구분']).mean().reset_index()
weekday_seungcha_y = weekday_avg_y[weekday_avg_y['승하차구분'] == '승차']
weekday_hacha_y = weekday_avg_y[weekday_avg_y['승하차구분'] == '하차']

weekends = ['2023-10-21', '2023-10-22']
weekend_y = new_rawy[new_rawy['수송일자'].isin(weekends)]
weekend_avg_y = weekend_y.groupby(['name', '승하차구분']).mean().reset_index()
weekend_seungcha_y = weekend_avg_y[weekend_avg_y['승하차구분'] == '승차']
weekend_hacha_y = weekend_avg_y[weekend_avg_y['승하차구분'] == '하차']

C:\Users\Chulwoong Park\AppData\Local\Temp\ipykernel_19756\3892198869.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  weekday_avg_y = weekday_y.groupby(['name', '승하차구분']).mean().reset_index()
C:\Users\Chulwoong Park\AppData\Local\Temp\ipykernel_19756\3892198869.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  weekend_avg_y = weekend_y.groupby(['name', '승하차구분']).mean().reset_index()


In [207]:
weekday_seungcha = raw_x.merge(weekday_seungcha_y, on = 'name')
weekday_hacha = raw_x.merge(weekday_hacha_y, on = 'name')
weekend_seungcha = raw_x.merge(weekend_seungcha_y, on = 'name')
weekend_hacha = raw_x.merge(weekend_hacha_y, on = 'name')

weekday_seungcha['weekend'] = 0
weekday_hacha['weekend'] = 0
weekend_seungcha['weekend'] = 1
weekend_hacha['weekend'] = 1

weekday_seungcha['hacha'] = 0
weekday_hacha['hacha'] = 1
weekend_seungcha['hacha'] = 0
weekend_hacha['hacha'] = 1

dataset = pd.concat([weekday_seungcha, weekday_hacha, weekend_seungcha, weekend_hacha])
dataset.to_csv('../Data/raw_dataset.csv')

In [210]:
dataset.reset_index(inplace = True)

In [211]:
# Get Max
dataset['max_ride'] = 0
dataset['min_ride'] = 0

for k, v in dataset.iterrows():
    dataset.iloc[k, -2] = v[['06시이전', '06-07시간대',
       '07-08시간대', '08-09시간대', '09-10시간대', '10-11시간대', '11-12시간대', '12-13시간대',
       '13-14시간대', '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대',
       '19-20시간대', '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대',    '24시이후']].max()
    dataset.iloc[k, -1] = v[['06시이전', '06-07시간대',
       '07-08시간대', '08-09시간대', '09-10시간대', '10-11시간대', '11-12시간대', '12-13시간대',
       '13-14시간대', '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대',
       '19-20시간대', '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대',    '24시이후']].min()
dataset.head(3)

,index,id,1,2,name,4,5,6,7,8,...,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후,weekend,hacha,max_ride,min_ride
0,0,6,01010000203B14000060DC46231D7D2D41F0384557DFA8...,5,가락시장,1,1,0,0,944586.7675563124951,...,977.4,795.6,730.8,603.8,250.6,63.4,0,0,2352.0,63.4
1,1,7,01010000203B140000D8ACFABCF0782D4108C58F81CFAB...,6,송파,0,1,0,0,1238646.417514794908,...,367.6,257.4,190.0,165.0,65.0,12.0,0,0,1710.6,12.0
2,2,8,01010000203B1400007836AB1EB67F2D41E0718A1E0FA6...,7,문정,0,1,0,0,595029.3087675537361,...,1828.4,1190.6,959.0,739.2,275.2,53.8,0,0,5700.4,53.8


In [221]:
# Get standardized
time_columns = ['06시이전', '06-07시간대',
       '07-08시간대', '08-09시간대', '09-10시간대', '10-11시간대', '11-12시간대', '12-13시간대',
       '13-14시간대', '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대',
       '19-20시간대', '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대',    '24시이후']

for k, v in dataset.iterrows():
    vols = v[time_columns].tolist()
    vmax, vmin = v['max_ride'], v['min_ride']
    normed_vols = [((i - vmin) / (vmax - vmin)) for i in vols]
    dataset.at[k, 'pattern'] = str(normed_vols)

In [224]:
#dataset_y = 
drop_cols = [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, '승하차구분'] + time_columns
dataset_y = dataset.drop(columns = drop_cols)

In [228]:
dataset_y.to_csv('../Data/raw_y.csv')

In [57]:
october_list = ['2023-10-'+str(d).zfill(2) for d in range(31)]

weekend_list = ['01', '02', '03', '09', '07', '08', '14', '15', '21', '22', '28', '29']
weekend_october = ['2023-10-'+d for d in weekend_list]

october_df = new_rawy[new_rawy['수송일자'].isin(october_list)]

weekend_y = october_df[october_df['수송일자'].isin(weekend_october)]
weekend_seungcha_y = weekend_y[weekend_y['승하차구분'] == '승차']
weekend_hacha_y = weekend_y[weekend_y['승하차구분'] == '하차']


weekday_y = october_df[~october_df['수송일자'].isin(weekend_october)]
weekday_seungcha_y = weekday_y[weekday_y['승하차구분'] == '승차']
weekday_hacha_y = weekday_y[weekday_y['승하차구분'] == '하차']

In [53]:
weekday_seungcha

,id,1,2,name,4,5,6,7,8,9,...,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후,weekend,hacha
0,8,01010000203B1400007836AB1EB67F2D41E0718A1E0FA6...,7,문정,0,1,0,0,595029.3087675537361,10703.90254576616,...,424,357,298,297,278,308,114.0,0,0,0
1,8,01010000203B1400007836AB1EB67F2D41E0718A1E0FA6...,7,문정,0,1,0,0,595029.3087675537361,10703.90254576616,...,3957,6317,1972,1120,858,571,255.0,51,0,0
2,8,01010000203B1400007836AB1EB67F2D41E0718A1E0FA6...,7,문정,0,1,0,0,595029.3087675537361,10703.90254576616,...,3548,5774,1953,1214,1128,747,266.0,55,0,0
3,8,01010000203B1400007836AB1EB67F2D41E0718A1E0FA6...,7,문정,0,1,0,0,595029.3087675537361,10703.90254576616,...,4054,5628,1750,1014,891,678,285.0,61,0,0
4,8,01010000203B1400007836AB1EB67F2D41E0718A1E0FA6...,7,문정,0,1,0,0,595029.3087675537361,10703.90254576616,...,3708,5880,1992,1272,1006,675,256.0,45,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4308,293,01010000203B140000FD1A2899BCDC2C417C1CD5C398B7...,292,양평,0,0,0,1,513431.08481943231531,55311.8524163867983,...,734,1421,377,204,191,118,52.0,13,0,0
4309,293,01010000203B140000FD1A2899BCDC2C417C1CD5C398B7...,292,양평,0,0,0,1,513431.08481943231531,55311.8524163867983,...,759,1458,351,220,200,136,52.0,15,0,0
4310,293,01010000203B140000FD1A2899BCDC2C417C1CD5C398B7...,292,양평,0,0,0,1,513431.08481943231531,55311.8524163867983,...,736,1451,329,267,176,127,75.0,15,0,0
4311,293,01010000203B140000FD1A2899BCDC2C417C1CD5C398B7...,292,양평,0,0,0,1,513431.08481943231531,55311.8524163867983,...,790,1394,377,207,164,147,88.0,16,0,0


In [58]:
weekday_seungcha = raw_x.merge(weekday_seungcha_y, on = 'name')
weekday_hacha = raw_x.merge(weekday_hacha_y, on = 'name')
weekend_seungcha = raw_x.merge(weekend_seungcha_y, on = 'name')
weekend_hacha = raw_x.merge(weekend_hacha_y, on = 'name')

weekday_seungcha['weekend'] = 0
weekday_hacha['weekend'] = 0
weekend_seungcha['weekend'] = 1
weekend_hacha['weekend'] = 1

weekday_seungcha['hacha'] = 0
weekday_hacha['hacha'] = 1
weekend_seungcha['hacha'] = 0
weekend_hacha['hacha'] = 1

dataset = pd.concat([weekday_seungcha, weekday_hacha, weekend_seungcha, weekend_hacha])



In [59]:
weekday_seungcha.to_csv('../Data/october_riding_wd_sc.csv')
weekday_hacha.to_csv('../Data/october_riding_wd_hc.csv')
weekend_seungcha.to_csv('../Data/october_riding_we_sc.csv')
weekend_hacha.to_csv('../Data/october_riding_we_hc.csv')

In [56]:
weekday_seungcha

,id,1,2,name,4,5,6,7,8,9,...,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후,weekend,hacha
0,8,01010000203B1400007836AB1EB67F2D41E0718A1E0FA6...,7,문정,0,1,0,0,595029.3087675537361,10703.90254576616,...,424,357,298,297,278,308,114.0,0,0,0
1,8,01010000203B1400007836AB1EB67F2D41E0718A1E0FA6...,7,문정,0,1,0,0,595029.3087675537361,10703.90254576616,...,3957,6317,1972,1120,858,571,255.0,51,0,0
2,8,01010000203B1400007836AB1EB67F2D41E0718A1E0FA6...,7,문정,0,1,0,0,595029.3087675537361,10703.90254576616,...,3548,5774,1953,1214,1128,747,266.0,55,0,0
3,8,01010000203B1400007836AB1EB67F2D41E0718A1E0FA6...,7,문정,0,1,0,0,595029.3087675537361,10703.90254576616,...,4054,5628,1750,1014,891,678,285.0,61,0,0
4,8,01010000203B1400007836AB1EB67F2D41E0718A1E0FA6...,7,문정,0,1,0,0,595029.3087675537361,10703.90254576616,...,3708,5880,1992,1272,1006,675,256.0,45,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4308,293,01010000203B140000FD1A2899BCDC2C417C1CD5C398B7...,292,양평,0,0,0,1,513431.08481943231531,55311.8524163867983,...,734,1421,377,204,191,118,52.0,13,0,0
4309,293,01010000203B140000FD1A2899BCDC2C417C1CD5C398B7...,292,양평,0,0,0,1,513431.08481943231531,55311.8524163867983,...,759,1458,351,220,200,136,52.0,15,0,0
4310,293,01010000203B140000FD1A2899BCDC2C417C1CD5C398B7...,292,양평,0,0,0,1,513431.08481943231531,55311.8524163867983,...,736,1451,329,267,176,127,75.0,15,0,0
4311,293,01010000203B140000FD1A2899BCDC2C417C1CD5C398B7...,292,양평,0,0,0,1,513431.08481943231531,55311.8524163867983,...,790,1394,377,207,164,147,88.0,16,0,0


In [32]:
dataset.to_csv('../Data/october_riding.csv')

In [37]:
weekday_seungcha.columns

Index([      'id',          1,          2,     'name',          4,          5,
                6,          7,          8,          9,         10,         11,
               12,         13,         14,         15,         16,         17,
               18,         19,     '수송일자',    '승하차구분',    '06시이전', '06-07시간대',
       '07-08시간대', '08-09시간대', '09-10시간대', '10-11시간대', '11-12시간대', '12-13시간대',
       '13-14시간대', '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대',
       '19-20시간대', '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대',    '24시이후',
        'weekend',    'hacha'],
      dtype='object')

In [52]:
test = weekday_seungcha[(weekday_seungcha['name']=='태릉입구')]
time_columns = ['06시이전', '06-07시간대',
       '07-08시간대', '08-09시간대', '09-10시간대', '10-11시간대', '11-12시간대', '12-13시간대',
       '13-14시간대', '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대',
       '19-20시간대', '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대',    '24시이후']
test[time_columns]

,06시이전,06-07시간대,07-08시간대,08-09시간대,09-10시간대,10-11시간대,11-12시간대,12-13시간대,13-14시간대,14-15시간대,15-16시간대,16-17시간대,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후
931,204,207,309,496,553,611,632,674,734,737,692,739,663,628,440,388,339,279,132.0,5
932,528,732,2018,2173,1127,708,703,763,783,745,786,855,1022,1168,727,432,433,354,179.0,55
933,525,713,1932,2212,1171,786,648,786,698,672,791,960,1096,1211,733,468,463,380,171.0,47
934,523,695,1853,2175,1139,772,727,854,767,784,979,979,1130,1230,710,501,429,370,212.0,60
935,536,758,1985,2264,1123,829,686,757,705,702,890,1015,1107,1310,716,526,442,354,164.0,54
936,520,713,1923,2292,1208,829,719,945,801,791,860,969,1163,1265,786,539,456,363,181.0,62
937,512,714,1899,2237,1228,835,700,784,683,735,854,1065,1167,1172,670,556,455,359,178.0,44
938,509,659,1822,2236,1163,800,675,896,812,849,1060,1007,1174,1312,701,483,466,428,215.0,57
939,520,747,1895,2231,1183,766,638,725,691,695,801,977,1133,1150,670,438,432,359,175.0,37
940,525,707,1975,2270,1195,799,741,839,735,755,924,1014,1200,1356,654,501,458,379,155.0,50
